# Download GNSS data

## Tools

*Modules*

In [1]:
import numpy as np
import pandas as pd
import requests

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
import matplotlib.cm as cmx
import matplotlib.ticker as ti
from matplotlib import patheffects
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
import cmcrameri.cm as cmc

import cartopy.crs as ccrs
import cartopy.feature as cf
import pygmt
from pyproj import Transformer
from pygmt.datasets import load_earth_relief
from fontTools.ttLib import TTFont
from matplotlib.font_manager import fontManager, FontProperties

from adjustText import adjust_text

In [2]:
plt.style.use('myfig.mplstyle')

*Paths*

In [3]:
# paths
wkdir = "/Users/emile/Documents/Etude/2024_2025_M2/tutored_project"
datadir = "{:}/data".format(wkdir)
plotdir = "{:}/figures".format(wkdir)

*Import list stations NGL*

In [4]:
gnss_stations = pd.read_csv("{:}/GNSS/list_stations_NGL.txt".format(datadir),
                            sep=' ', skipinitialspace=True, names=['code', 'lat', 'lon', 'height'], usecols=[0, 1, 2, 3])
gnss_stations.loc[gnss_stations.lon <= -180, 'lon'] += 360

*Projections*

In [5]:
crs_geo = 'EPSG:4326'
crs_utm = 'EPSG:32636'

## Select stations

In [6]:
lonmin, lonmax = 34.5, 36.5
latmin, latmax = 30, 32.5

In [7]:
gnss_stations_red = gnss_stations[((gnss_stations.lat >= latmin) & (gnss_stations.lat <= latmax)) & ((gnss_stations.lon >= lonmin) & (gnss_stations.lon <= lonmax))]
gnss_stations_red.reset_index(drop=True, inplace=True)

In [8]:
transformer = Transformer.from_crs(crs_geo, crs_utm)
coords_utm = transformer.transform(gnss_stations_red.lat, gnss_stations_red.lon)
x_utm = coords_utm[0]
y_utm = coords_utm[1]

In [9]:
gnss_stations_red['x'] = x_utm
gnss_stations_red['y'] = y_utm

/var/folders/58/4tzv12sd173350mg97rbrhwc0000gn/T/ipykernel_8523/1654124809.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_stations_red['x'] = x_utm
/var/folders/58/4tzv12sd173350mg97rbrhwc0000gn/T/ipykernel_8523/1654124809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gnss_stations_red['y'] = y_utm


In [10]:
gnss_stations_red.to_csv("{:}/GNSS/list_stations_dead_sea.txt".format(datadir),
                         sep='\t', index=False)

In [13]:
for idx, row in gnss_stations_red.iterrows():
    
    # IGS 14
    
    url = "http://geodesy.unr.edu/gps_timeseries/tenv3/IGS14/{:}.tenv3".format(row.code)
    response = requests.get(url, allow_redirects=True)
    
    with open("{:}/GNSS/{:}.tenv3".format(datadir, row.code), "w") as f:
        f.write(response.text)
    
    # African plate reference frame
    
    url = "http://geodesy.unr.edu/gps_timeseries/tenv3/plates/AF/{:}.AF.tenv3".format(row.code)
    response = requests.get(url, allow_redirects=True)
    
    with open("{:}/GNSS/{:}.AF.tenv3".format(datadir, row.code), "w") as f:
        f.write(response.text)
        
    # Arabian plate reference frame
    
    url = "http://geodesy.unr.edu/gps_timeseries/tenv3/plates/AR/{:}.AR.tenv3".format(row.code)
    response = requests.get(url, allow_redirects=True)
    
    with open("{:}/GNSS/{:}.AR.tenv3".format(datadir, row.code), "w") as f:
        f.write(response.text)